In [0]:
%restart_python

In [0]:
import os 
from pyspark.dbutils import DBUtils 
d = DBUtils(spark)  
os.environ["API_KEY"] = d.secrets.get(scope="faical-scope", key="API_KEY")


In [0]:
import requests
import os

API_KEY = os.environ["API_KEY"]

url = "https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/eco2mix-metropoles-tr/records"

headers = {
    "Authorization": f"Apikey {API_KEY}",
    "Accept": "application/json",
}

all_rows = []
offset = 0

while True:
    params = {
        "timezone": "Europe/Paris",
        "offset": offset,
        "order_by": "date_heure ASC",
        "where": "date_heure >= '2020-01-28' AND date_heure <= '2025-11-28'",
    }

    resp = requests.get(url, headers=headers, params=params)
    batch = resp.json().get("results", [])

    print("Fetching:", resp.url)

    if not batch:
        break

    all_rows.extend(batch)
    offset += 20  # default page size is 20

print("Total fetched:", len(all_rows))


In [0]:
%sql
DROP TABLE IF EXISTS eco2mix_metropoles_tr;


In [0]:
#all_rows

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, LongType

schema = StructType([
    StructField("code_insee_epci",   StringType(), True), # Code métropole
    StructField("libelle_metropole", StringType(), True), # Code métropole
    StructField("nature",            StringType(), True), # real time
    StructField("date",              StringType(), True), # YYYY-MM-DD
    StructField("heures",            StringType(), True), # HH:mm
    StructField("date_heure",        StringType(), True), # can cast to Timestamp later
    StructField("consommation",      LongType(),   True), # in MW
    StructField("production",        LongType(),   True), 
    StructField("echanges_physiques",LongType(),   True),
])

df = spark.createDataFrame(all_rows, schema=schema)
display(df)


Cast date & datetime 

In [0]:
from pyspark.sql.functions import to_date, to_timestamp

df_clean = (
    df
    .withColumn("date", to_date("date", "yyyy-MM-dd"))
    .withColumn("date_heure", to_timestamp("date_heure"))
    .drop("echanges_physiques", "production")
)
display(df_clean)


In [0]:
df_clean.write.format("delta").mode("append").saveAsTable("eco2mix_metropoles_tr")

In [0]:
df_sql = spark.table("eco2mix_metropoles_tr")
display(df_sql)

In [0]:
print("h")